# MapReduce Graduaciones

In [1]:
###############################################################
#!/usr/bin/python
# Purpose: Migrate Products from MySQL to Mongo
# Dependecies : python-pip , pymongo , mysqldb
# apt-get install python-pip ; pip install pymongo;
# apt-get install python-MySQLdb


import pymongo
import datetime
import enum
from pprint import pprint

In [2]:
mongodb_host = "mongodb://localhost:11111/"
mongodb_dbname = "project_test"

mclient = pymongo.MongoClient(mongodb_host)
mdb = mclient[mongodb_dbname] 

In [3]:
collections = mdb.list_collection_names()
print(collections)

['contrataciones', 'registro_publicaciones', 'egresado', 'revista', 'cargos', 'sede_universidad', 'time_', 'publicacion', 'departamento', 'programa', 'institucion', 'facultad', 'graduaciones', 'empresa']


In [4]:
graduaciones = mdb["graduaciones"]
pprint(graduaciones.find()[0])

{'_id': ObjectId('6621a41fb3c7f9a39bdf6f57'),
 'date_id': datetime.datetime(2015, 11, 15, 0, 0),
 'departamento_id': {'departamento_id': 10,
                     'departamento_nombre': 'DEPARTAMENTO DE PSICOLOGÍA'},
 'egresado_id': {'egresado_address': '03403 Tyler Neck Apt. 540\n'
                                     'Lake Patrick, GA 66352',
                 'egresado_cellphone': '(837)862-1',
                 'egresado_edad': 37,
                 'egresado_emailaddress': 'thomaspatrick@gmail.com',
                 'egresado_etnicgroup': 'ACTIVO',
                 'egresado_firstlastname': 'COHEN',
                 'egresado_gender': 'MASCULINO',
                 'egresado_id': 1,
                 'egresado_names': 'RICHARD',
                 'egresado_secondlastname': 'PINEDA'},
 'facultad_id': {'facultad_id': 11, 'facultad_nombre': 'ODONTOLOGÍA'},
 'programa_id': {'programa_id': 26,
                 'programa_nivelacademico': 'PREGRADO',
                 'programa_nombre': 'INGENIE

In [5]:
# Egresados por facultad

stage_group_facultad = {
  "$group": {
    "_id" : "$facultad_id.facultad_id",
    "count_egresado": {"$sum": 1}
  }
}

stage_out_facultad = {
  "$out": "graduaciones_facultad"
}

pipeline = [
  stage_group_facultad,
  stage_out_facultad
]

results = graduaciones.aggregate(pipeline)

for graduacion_mapreduce in mdb["graduaciones_facultad"].find():
  print(graduacion_mapreduce)

{'_id': 2, 'count_egresado': 17}
{'_id': 9, 'count_egresado': 17}
{'_id': 4, 'count_egresado': 6}
{'_id': 6, 'count_egresado': 16}
{'_id': 10, 'count_egresado': 15}
{'_id': 1, 'count_egresado': 2}
{'_id': 8, 'count_egresado': 5}
{'_id': 3, 'count_egresado': 3}
{'_id': 11, 'count_egresado': 19}


In [6]:
# Egresados por tipo programa

stage_group_tipo_programa = {
  "$group": {
    "_id" : "$programa_id.programa_nivelacademico",
    "count_egresado": {"$sum": 1}
  }
}

stage_out_tipo_programa = {
  "$out": "graduaciones_tipo_programa"
}

pipeline = [
  stage_group_tipo_programa,
  stage_out_tipo_programa
]

results = graduaciones.aggregate(pipeline)

for graduacion_mapreduce in mdb["graduaciones_tipo_programa"].find():
  print(graduacion_mapreduce)

{'_id': 'POSGRADO', 'count_egresado': 79}
{'_id': 'PREGRADO', 'count_egresado': 21}


In [7]:
# Egresados por tipo sede

stage_group_sede = {
  "$group": {
    "_id" : "$sede_id",
    "count_egresado": {"$sum": 1}
  }
}

stage_out_sede = {
  "$out": "graduaciones_sede"
}

pipeline = [
  stage_group_sede,
  stage_out_sede
]

results = graduaciones.aggregate(pipeline)

for graduacion_mapreduce in mdb["graduaciones_sede"].find():
  print(graduacion_mapreduce)

{'_id': {'sede_nombre': 'CARIBE', 'sede_id': 2}, 'count_egresado': 14}
{'_id': {'sede_nombre': 'MANIZALES', 'sede_id': 4}, 'count_egresado': 14}
{'_id': {'sede_nombre': 'AMAZONIA', 'sede_id': 1}, 'count_egresado': 22}
{'_id': {'sede_nombre': 'TUMACO Y DE LA PAZ', 'sede_id': 8}, 'count_egresado': 11}
{'_id': {'sede_nombre': 'MEDELLÍN', 'sede_id': 5}, 'count_egresado': 15}
{'_id': {'sede_nombre': 'ORINOQUIA', 'sede_id': 6}, 'count_egresado': 14}
{'_id': {'sede_nombre': 'BOGOTÁ', 'sede_id': 3}, 'count_egresado': 10}


In [8]:
# Egresados por tipo sede

stage_group_year = {
  "$group": {
    "_id" : {"$year": {"$toDate": "$date_id"}},
    "count_egresado": {"$sum": 1}
  }
}

stage_out_year = {
  "$out": "graduaciones_year"
}

pipeline = [
  stage_group_year,
  stage_out_year
]

results = graduaciones.aggregate(pipeline)

for graduacion_mapreduce in mdb["graduaciones_year"].find():
  print(graduacion_mapreduce)

{'_id': 1981, 'count_egresado': 2}
{'_id': 2012, 'count_egresado': 1}
{'_id': 1974, 'count_egresado': 2}
{'_id': 2021, 'count_egresado': 2}
{'_id': 2000, 'count_egresado': 1}
{'_id': 1988, 'count_egresado': 3}
{'_id': 1999, 'count_egresado': 2}
{'_id': 1992, 'count_egresado': 1}
{'_id': 1971, 'count_egresado': 2}
{'_id': 2008, 'count_egresado': 4}
{'_id': 1998, 'count_egresado': 2}
{'_id': 1977, 'count_egresado': 1}
{'_id': 1980, 'count_egresado': 1}
{'_id': 1995, 'count_egresado': 1}
{'_id': 2016, 'count_egresado': 6}
{'_id': 1982, 'count_egresado': 1}
{'_id': 1978, 'count_egresado': 3}
{'_id': 2019, 'count_egresado': 3}
{'_id': 1979, 'count_egresado': 3}
{'_id': 1991, 'count_egresado': 2}
{'_id': 1984, 'count_egresado': 2}
{'_id': 2022, 'count_egresado': 5}
{'_id': 2005, 'count_egresado': 1}
{'_id': 2004, 'count_egresado': 1}
{'_id': 2010, 'count_egresado': 1}
{'_id': 2011, 'count_egresado': 3}
{'_id': 2009, 'count_egresado': 3}
{'_id': 1983, 'count_egresado': 2}
{'_id': 1996, 'count